# PP3: Local search demos


We demonstrate four local search algorithms: (random-restart) Hill climbing, simulated annealing, local beam search and genetic algorithm, on n-queens problem.

### Exercise 0 (warm-up) 
Solve n-queens problem with BFS. 

- **State Representation**:
Each state is a list of integers, where the index represents the column, and the value at that index represents the row position of the queen.
Example: [1, 3, 0, 2] means a queen is at (0,1), (1,3), (2,0), and (3,2).
- **BFS Approach**:
Start with an empty board and add queens column by column.
At each level, the algorithm will try placing a queen in each row of the current column.
If a state has queens in all columns and there are no conflicts, it's considered a solution.
For each state, if depth < n, it tries adding a queen in every possible row for the next column.
Only valid moves (no conflicts) are added to the queue for further exploration.
- **Termination**:
The algorithm terminates when it finds a complete, valid state with n queens, or if the queue is exhausted (indicating no solution was found).
- **Verbose Logging**:
The verbose parameter controls whether detailed output is shown.
The log prints the current state, depth, and new valid moves added to the queue.


In [30]:
from collections import deque
import time

def is_valid_state(state):
    col = len(state) - 1
    row = state[-1]
    
    for c in range(col):
        if abs(state[c] - row) == abs(col - c) or state[c] == row:
            return False
    return True

def print_board(state, n):
    board = [["." for _ in range(n)] for _ in range(n)]
    for col, row in enumerate(state):
        board[row][col] = "Q"
    
    for row in board:
        print(" ".join(row))
    print()

def bfs_n_queens(n=4, time_start = -1, time_limit = 120, verbose=False):
    # Queue stores (state, depth), where `state` is the current configuration of queens
    # and `depth` is the current number of queens placed
    queue = deque([([], 0)])
    steps = 0
    start_time = time.time()
    
    while queue:
        state, depth = queue.popleft()
        steps += 1
        
        if verbose:
            print(f"Processing state at depth {depth}: {state}")
            print_board(state, n)
        
        if time_start != - 1:
            if abs(time_start - time.time()) > time_limit:
                return None
        
        # Check if the solution is found
        if depth == n:
            end_time = time.time()
            runtime = start_time - end_time
            if verbose:
                print(f"Solution found: {state}")
                print(f"Steps taken: {steps}")
                print(f"Runtime: {runtime:.4f} seconds")
            return state, steps, runtime
        
        # Expand the current state by placing a queen in the next column
        for row in range(n):
            new_state = state + [row]
            if is_valid_state(new_state):
                if verbose:
                    print(f"  Valid move: Adding queen at column {depth}, row {row} -> New state: {new_state}")
                    print_board(new_state, n)
                queue.append((new_state, depth + 1))
    
    if verbose:
        print("No solution found.")
    return None, steps, time.time() - start_time

def print_solution(state):
    if state is None:
        print("No solution found")
        return
    
    n = len(state)
    print_board(state, n)

# Solve the 4-queens problem using BFS with verbose logging
solution, steps, runtime = bfs_n_queens(n=2, verbose=True)
print_solution(solution)
print(f"Total steps: {steps}")
print(f"Total runtime: {runtime:.4f} seconds")


Processing state at depth 0: []
. .
. .

  Valid move: Adding queen at column 0, row 0 -> New state: [0]
Q .
. .

  Valid move: Adding queen at column 0, row 1 -> New state: [1]
. .
Q .

Processing state at depth 1: [0]
Q .
. .

Processing state at depth 1: [1]
. .
Q .

No solution found.
No solution found
Total steps: 3
Total runtime: 0.0010 seconds


### Exercise 0.2
Print all solutions for 4-queens with BFS. 

In [31]:
from collections import deque
import time

def is_valid_state(state):
    col = len(state) - 1
    row = state[-1]
    
    for c in range(col):
        if state[c] == row or abs(state[c] - row) == abs(col - c):
            return False
    return True

def print_board(state, n):
    board = [["." for _ in range(n)] for _ in range(n)]
    for col, row in enumerate(state):
        board[row][col] = "Q"
    
    for row in board:
        print(" ".join(row))
    print()

def bfs_n_queens_all_solutions(n=4, verbose=False):
    queue = deque([([], 0)])
    steps = 0
    solutions = []
    start_time = time.time()
    
    while queue:
        state, depth = queue.popleft()
        steps += 1
        
        if verbose:
            print(f"Processing state at depth {depth}: {state}")
        
        # Check if the solution is found
        if depth == n:
            solutions.append(state)
            if verbose:
                print(f"Solution found: {state}")
                print_board(state, n)
        
        # Expand the current state by placing a queen in the next column
        for row in range(n):
            new_state = state + [row]
            if is_valid_state(new_state):
                queue.append((new_state, depth + 1))
    
    end_time = time.time()
    runtime = end_time - start_time
    
    return solutions, steps, runtime

def print_all_solutions(solutions):
    if not solutions:
        print("No solutions found")
        return
    
    for idx, solution in enumerate(solutions, 1):
        print(f"Solution {idx}:")
        print_board(solution, len(solution))

# Solve the 4-queens problem using BFS to find all solutions
solutions, steps, runtime = bfs_n_queens_all_solutions(n=4, verbose=False)
print_all_solutions(solutions)
print(f"Total solutions found: {len(solutions)}")
print(f"Total steps: {steps}")
print(f"Total runtime: {runtime:.4f} seconds")


Solution 1:
. . Q .
Q . . .
. . . Q
. Q . .

Solution 2:
. Q . .
. . . Q
Q . . .
. . Q .

Total solutions found: 2
Total steps: 17
Total runtime: 0.0000 seconds


### Exercise 0.3 
Report the percentage of configurations which are solutions.

In [32]:
from collections import deque
import time

def is_valid_state_all(state):
    conflicts = 0
    n = len(state)
    for col1 in range(n):
        for col2 in range(col1 + 1, n):
            row1, row2 = state[col1], state[col2]
            if row1 == row2 or col1 == col2 or abs(row1 - row2) == abs(col1 - col2):
                conflicts += 1
    return conflicts == 0

def bfs_proportion_solutions(n=4):
    # Queue stores (state, depth), where `state` is the current configuration of queens
    # and `depth` is the current number of queens placed
    queue = deque([([], 0)])
    total_configurations = 0
    solutions_count = 0
    start_time = time.time()
    
    while queue:
        state, depth = queue.popleft()
        
        # If a configuration is complete, count it as a total configuration
        if depth == n:
            total_configurations += 1
            if is_valid_state_all(state):
                print_board(state, n)
                solutions_count += 1
            continue
        
        # Expand the current state by placing a queen in the next column
        for row in range(n):
            new_state = state + [row]
            queue.append((new_state, depth + 1))
    
    end_time = time.time()
    runtime = start_time - end_time
    
    return solutions_count, total_configurations, runtime

# Run the BFS to find the proportion of solutions
solutions_count, total_configurations, runtime = bfs_proportion_solutions(n=4)
proportion = solutions_count / total_configurations if total_configurations > 0 else 0

print(f"Total configurations: {total_configurations}")
print(f"Total solutions: {solutions_count}")
print(f"Proportion of solutions: {proportion:.6f}")
print(f"Total runtime: {runtime:.4f} seconds")


. . Q .
Q . . .
. . . Q
. Q . .

. Q . .
. . . Q
Q . . .
. . Q .

Total configurations: 256
Total solutions: 2
Proportion of solutions: 0.007812
Total runtime: -0.0010 seconds


### Exercise 1.1 (Hill climbing) 
Use local search with Hill climbing to find one solution. 

-**State Representation**:
The state is a list where the index represents the column, and the value represents the row position of the queen in that column.
Example: [1, 3, 0, 2] indicates queens placed at (0,1), (1,3), (2,0), and (3,2).

-**Hill Climbing Approach**:
Start with a random initial state.
Evaluate all possible moves for each queen (moving it to any row in its column).
Select the move that reduces conflicts the most.
Repeat until no better move is found or a solution is achieved.

-**Verbose Logging**:
The algorithm will print the current board configuration, the conflicts count, and the steps it takes.
This helps track how the algorithm is trying to improve the configuration.

In [33]:
import random
import time

def calculate_conflicts(state):
    """Calculate the number of conflicts (attacking pairs) in the current state."""
    conflicts = 0
    n = len(state)
    for col1 in range(n):
        for col2 in range(col1 + 1, n):
            row1, row2 = state[col1], state[col2]
            if row1 == row2 or col1 == col2 or abs(row1 - row2) == abs(col1 - col2):
                conflicts += 1
    return conflicts

def generate_neighbor(state):
    """Generate a neighboring state by moving a queen in a random column to a new row."""
    n = len(state)
    col = random.randint(0, n - 1)
    current_row = state[col]
    
    # Choose a new row for the queen in the selected column (different from the current row)
    new_row = random.choice([r for r in range(n) if r != current_row])
    
    # Create a new state with the modified position
    new_state = state[:]
    new_state[col] = new_row
    
    return new_state

def print_board(state, n):
    board = [["." for _ in range(n)] for _ in range(n)]
    for col, row in enumerate(state):
        board[row][col] = "Q"
    
    for row in board:
        print(" ".join(row))
    print()

def hill_climbing_n_queens(n=4, time_start = -1, time_limit = 120, verbose=False):
    # Start with a random configuration
    current_state = [random.randint(0, n - 1) for _ in range(n)]
    current_conflicts = calculate_conflicts(current_state)
    steps = 0
    start_time = time.time()
    
    while current_conflicts > 0:
        steps += 1
        if verbose:
            print(f"Step {steps}: Current state with {current_conflicts} conflicts")
            print_board(current_state, n)
        
        # Generate a list of all possible neighbors and find the one with the least conflicts
        best_neighbor = current_state
        best_conflicts = current_conflicts

        if time_start != - 1:
            if abs(time_start - time.time()) > time_limit:
                return None
        
        for _ in range(n * (n - 1)):  # Explore multiple possible moves to avoid local maxima
            neighbor = generate_neighbor(current_state)
            neighbor_conflicts = calculate_conflicts(current_state)
            
            if neighbor_conflicts < best_conflicts:
                best_neighbor = neighbor
                best_conflicts = neighbor_conflicts
        
        # If no improvement is possible, stop (local optimum)
        if best_conflicts >= current_conflicts:
            break
        
        # Move to the best neighbor found
        current_state = best_neighbor
        current_conflicts = best_conflicts
    
    end_time = time.time()
    runtime = end_time - start_time
    
    return current_state, steps, runtime, current_conflicts == 0

# Solve the 4-queens problem using Hill Climbing
solution, steps, runtime, found = hill_climbing_n_queens(n=4, verbose=True)

if found:
    print("Solution found:")
    print_board(solution, len(solution))
else:
    print("No solution found.")
print(f"Total steps: {steps}")
print(f"Total runtime: {runtime:.4f} seconds")


Step 1: Current state with 4 conflicts
Q . . .
. . . .
. Q Q .
. . . Q

No solution found.
Total steps: 1
Total runtime: 0.0010 seconds


### Exercise 1.2 (Hill climbing with random restarts)
Add the random restart phase. 
1. Test your algorithm when starts from a diagonal solution, and when it starts from a randomized solution. 
(Answer: the one starting with the diagonal solution does not find a solution -> in general, simple Hill climbing is incomplete)
2. In how many cases does the randomised start give a solution? 
(Answer: it should be 0.14 for the case of n=8) 
3. Test now the randomised start Hill climbing program for n from 1 to 10. Report the time to get the solution in a table.

In [34]:
import random
import time

def calculate_conflicts(state):
    """Calculate the number of conflicts (attacking pairs) in the current state."""
    conflicts = 0
    n = len(state)
    for col1 in range(n):
        for col2 in range(col1 + 1, n):
            row1, row2 = state[col1], state[col2]
            if row1 == row2 or col1 == col2 or abs(row1 - row2) == abs(col1 - col2):
                conflicts += 1
    return conflicts

def generate_neighbor(state):
    """Generate a neighboring state by moving a queen in a random column to a new row."""
    n = len(state)
    col = random.randint(0, n - 1)
    current_row = state[col]
    
    # Choose a new row for the queen in the selected column (different from the current row)
    new_row = random.choice([r for r in range(n) if r != current_row])
    
    # Create a new state with the modified position
    new_state = state[:]
    new_state[col] = new_row
    
    return new_state

def hill_climbing_n_queens(n=4, time_start = -1, time_limit = 120, verbose=False):
    # Start with a uniform random permutation of queens
    current_state = list(range(n))
    random.shuffle(current_state)
    current_conflicts = calculate_conflicts(current_state)
    steps = 0
    start_time = time.time()
    
    while current_conflicts > 0:
        steps += 1
        if verbose:
            print(f"Step {steps}: Current state with {current_conflicts} conflicts")
        
        # Generate a list of all possible neighbors and find the one with the least conflicts
        best_neighbor = current_state
        best_conflicts = current_conflicts

        if time_start != - 1:
            if abs(time_start - time.time()) > time_limit:
                return None
        
        for _ in range(n * (n - 1)):  # Explore multiple possible moves to avoid local maxima
            neighbor = generate_neighbor(current_state)
            neighbor_conflicts = calculate_conflicts(neighbor)
            
            if neighbor_conflicts < best_conflicts:
                best_neighbor = neighbor
                best_conflicts = neighbor_conflicts
        
        # If no improvement is possible, stop (local optimum)
        if best_conflicts >= current_conflicts:
            break
        
        # Move to the best neighbor found
        current_state = best_neighbor
        current_conflicts = best_conflicts
    
    end_time = time.time()
    runtime = end_time - start_time
    
    return current_state, steps, runtime, current_conflicts == 0

def evaluate_hill_climbing(n=4, trials=100):
    successes = 0
    failures = 0
    total_steps_success = 0
    total_runtime_success = 0
    total_steps_failure = 0
    total_runtime_failure = 0
    
    for _ in range(trials):
        _, steps, runtime, found = hill_climbing_n_queens(n=n, verbose=False)
        
        if found:
            successes += 1
            total_steps_success += steps
            total_runtime_success += runtime
        else:
            failures += 1
            total_steps_failure += steps
            total_runtime_failure += runtime
    
    # Calculate averages
    avg_steps_success = total_steps_success / successes if successes > 0 else 0
    avg_runtime_success = total_runtime_success / successes if successes > 0 else 0
    avg_steps_failure = total_steps_failure / failures if failures > 0 else 0
    avg_runtime_failure = total_runtime_failure / failures if failures > 0 else 0
    
    print(f"Total runs: {trials}")
    print(f"Successful runs: {successes}")
    print(f"Failed runs: {failures}")
    if successes > 0:
        print(f"Average steps (success): {avg_steps_success:.2f}")
        print(f"Average runtime (success): {avg_runtime_success:.4f} seconds")
    if failures > 0:
        print(f"Average steps (failure): {avg_steps_failure:.2f}")
        print(f"Average runtime (failure): {avg_runtime_failure:.4f} seconds")

#Evaluate the Hill Climbing algorithm over 100 trials
evaluate_hill_climbing(n=8, trials=1000)


Total runs: 1000
Successful runs: 60
Failed runs: 940
Average steps (success): 3.43
Average runtime (success): 0.0027 seconds
Average steps (failure): 3.16
Average runtime (failure): 0.0024 seconds


### Exercise 2: Simulated annealing search
Implement the simulated annealing algorithm for solving the n-queens problem.
1. Which parameters will you have to introduce? Set: max_iterations=1000, initial_temperature=100, cooling_rate=0.99
2. Report if the solution was found, and the **average** time to find a solution in each case, varying n from 1 to 10, max_iterations to 100 and 10000 and decreasing the cooling_rate to 0.89 and 0.79. Report your observations and try to explain what you see.

In [35]:
import random
import math

def calculate_conflicts(state):
    conflicts = 0
    n = len(state)
    
    for col1 in range(n):
        row1 = state[col1]
        for col2 in range(col1 + 1, n):
            row2 = state[col2]
            if row1 == row2 or col1 == col2 or abs(row1 - row2) == abs(col1 - col2):
                conflicts += 1
    return conflicts

def print_board(state, n):
    board = [["." for _ in range(n)] for _ in range(n)]
    for col, row in enumerate(state):
        board[row][col] = "Q"
    
    for row in board:
        print(" ".join(row))
    print()

def random_initial_state(n):
    return [random.randint(0, n - 1) for _ in range(n)]

def probability_acceptance(delta_e, temperature):
    if delta_e < 0:
        return 1.0
    else:
        return math.exp(-delta_e / temperature)

def simulated_annealing_n_queens(n=4, max_iterations=1000, initial_temperature=100, cooling_rate=0.99, time_start = -1, time_limit = 120, verbose=False):
    current_state = random_initial_state(n)
    current_conflicts = calculate_conflicts(current_state)
    temperature = initial_temperature
    
    if verbose:
        print("Initial random state:")
        print_board(current_state, n)
        print(f"Initial conflicts: {current_conflicts}")
        print(f"Initial temperature: {temperature}\n")
    
    for iteration in range(max_iterations):
        # Terminate if a solution is found
        if current_conflicts == 0:
            if verbose:
                print(f"Solution found after {iteration + 1} iterations:")
                print_board(current_state, n)
            return current_state

        if time_start != - 1:
            if abs(time_start - time.time()) > time_limit:
                return None
        
        # Pick a random neighbor by moving one queen to a different row in its column
        col = random.randint(0, n - 1)
        new_state = current_state[:]
        new_state[col] = random.randint(0, n - 1)
        new_conflicts = calculate_conflicts(new_state)
        
        # Calculate change in conflicts
        delta_e = new_conflicts - current_conflicts
        
        # Decide whether to accept the new state
        accepted = False
        if probability_acceptance(delta_e, temperature) > random.random():
            current_state = new_state
            current_conflicts = new_conflicts
            accepted = True
        
        if verbose:
            print(f"Iteration {iteration + 1}:")
            print_board(current_state, n)
            print(f"Temperature: {temperature:.2f}")
            print(f"Delta conflicts (ΔE): {delta_e}")
            print(f"Conflicts: {current_conflicts}")
            print("Action: " + ("Accepted new state" if accepted else "Rejected new state"))
            print()
        
        # Cool down the temperature
        temperature *= cooling_rate
    
    if verbose:
        print("No solution found after maximum iterations.")
        print("Final state:")
        print_board(current_state, n)
    return None

# Solve the 4-queens problem using Simulated Annealing with verbose logging
solution = simulated_annealing_n_queens(n=4, max_iterations=1000, initial_temperature=100, cooling_rate=0.99, verbose=True)


Initial random state:
Q Q . .
. . Q .
. . . .
. . . Q

Initial conflicts: 3
Initial temperature: 100

Iteration 1:
. Q . .
Q . Q .
. . . .
. . . Q

Temperature: 100.00
Delta conflicts (ΔE): 0
Conflicts: 3
Action: Accepted new state

Iteration 2:
Q Q . .
. . Q .
. . . .
. . . Q

Temperature: 99.00
Delta conflicts (ΔE): 0
Conflicts: 3
Action: Accepted new state

Iteration 3:
. Q . .
. . Q .
Q . . .
. . . Q

Temperature: 98.01
Delta conflicts (ΔE): -2
Conflicts: 1
Action: Accepted new state

Iteration 4:
. Q . .
. . Q .
Q . . .
. . . Q

Temperature: 97.03
Delta conflicts (ΔE): 0
Conflicts: 1
Action: Accepted new state

Iteration 5:
. Q . .
. . Q .
Q . . Q
. . . .

Temperature: 96.06
Delta conflicts (ΔE): 3
Conflicts: 4
Action: Accepted new state

Iteration 6:
. Q . .
. . Q .
. . . Q
Q . . .

Temperature: 95.10
Delta conflicts (ΔE): 0
Conflicts: 4
Action: Accepted new state

Iteration 7:
. Q Q .
. . . .
. . . Q
Q . . .

Temperature: 94.15
Delta conflicts (ΔE): -2
Conflicts: 2
Action: Accep

### Exercise 3: Local beam search
Local beam search with the following parameters: *k=3*, *max_iterations=1000*. Report if the solution was found, and the average time to find a solution, for varying *n*, *k* and *max_iterations*. What do you observe?

In [36]:
import random

def calculate_conflicts(state):
    conflicts = 0
    n = len(state)
    
    for col1 in range(n):
        row1 = state[col1]
        for col2 in range(col1 + 1, n):
            row2 = state[col2]
            if row1 == row2 or col1 == col2 or abs(row1 - row2) == abs(col1 - col2):
                conflicts += 1
    return conflicts

def print_board(state, n):
    board = [["." for _ in range(n)] for _ in range(n)]
    for col, row in enumerate(state):
        board[row][col] = "Q"
    
    for row in board:
        print(" ".join(row))
    print()

def random_initial_state(n):
    return [random.randint(0, n - 1) for _ in range(n)]

def local_beam_search(n=4, k=3, max_iterations=1000, time_start = -1, time_limit = 120, verbose=False):
    # Step 1: Generate k random initial states (beams)
    beams = [random_initial_state(n) for _ in range(k)]
    
    if verbose:
        print("Initial random states:")
        for i, state in enumerate(beams):
            print(f"Beam {i + 1}:")
            print_board(state, n)
        print()
    
    for iteration in range(max_iterations):
        # Step 2: Calculate conflicts for all beams
        beam_conflicts = [(state, calculate_conflicts(state)) for state in beams]
        
        # Step 3: Check if any beam has zero conflicts (solution found)
        for state, conflicts in beam_conflicts:
            if conflicts == 0:
                if verbose:
                    print(f"Solution found after {iteration + 1} iterations:")
                    print_board(state, n)
                return state
        
        if time_start != - 1:
            if abs(time_start - time.time()) > time_limit:
                return None
        
        # Step 4: Select the best beams
        beam_conflicts.sort(key=lambda x: x[1])  # Sort by conflicts
        best_beams = [state for state, _ in beam_conflicts[:k]]  # Keep k best beams
        
        # Step 5: Generate new states from the best beams
        new_beams = []
        for state in best_beams:
            for col in range(n):
                for row in range(n):
                    if state[col] != row:  # Change the position of one queen
                        new_state = state[:]
                        new_state[col] = row
                        new_beams.append(new_state)
        
        # Step 6: Add new beams and limit to k
        beams = new_beams[:k]
        
        if verbose:
            print(f"Iteration {iteration + 1}:")
            print(f"Best beams after selecting:")
            for i, state in enumerate(best_beams):
                print(f"Best Beam {i + 1}:")
                print_board(state, n)
            print(f"New beams (next iteration):")
            for i, state in enumerate(beams):
                print(f"New Beam {i + 1}:")
                print_board(state, n)
            print()
    
    if verbose:
        print("No solution found after maximum iterations.")
    return None

# Solve the 4-queens problem using Local Beam Search with verbose logging
solution = local_beam_search(n=4, k=3, max_iterations=1000, verbose=True)


Initial random states:
Beam 1:
. Q Q .
. . . Q
. . . .
Q . . .

Beam 2:
. . . .
. . . .
. . Q Q
Q Q . .

Beam 3:
. . . .
Q . . Q
. Q Q .
. . . .


Iteration 1:
Best beams after selecting:
Best Beam 1:
. Q Q .
. . . Q
. . . .
Q . . .

Best Beam 2:
. . . .
. . . .
. . Q Q
Q Q . .

Best Beam 3:
. . . .
Q . . Q
. Q Q .
. . . .

New beams (next iteration):
New Beam 1:
Q Q Q .
. . . Q
. . . .
. . . .

New Beam 2:
. Q Q .
Q . . Q
. . . .
. . . .

New Beam 3:
. Q Q .
. . . Q
Q . . .
. . . .


Iteration 2:
Best beams after selecting:
Best Beam 1:
. Q Q .
. . . Q
Q . . .
. . . .

Best Beam 2:
Q Q Q .
. . . Q
. . . .
. . . .

Best Beam 3:
. Q Q .
Q . . Q
. . . .
. . . .

New beams (next iteration):
New Beam 1:
Q Q Q .
. . . Q
. . . .
. . . .

New Beam 2:
. Q Q .
Q . . Q
. . . .
. . . .

New Beam 3:
. Q Q .
. . . Q
. . . .
Q . . .


Iteration 3:
Best beams after selecting:
Best Beam 1:
. Q Q .
. . . Q
. . . .
Q . . .

Best Beam 2:
Q Q Q .
. . . Q
. . . .
. . . .

Best Beam 3:
. Q Q .
Q . . Q
. . .

### Exercise 4. Genetic algorithm

See the implementation of the genetic algorithm for n-queens problem, with parameters n=4, population_size=10, generations=1000, mutation_rate=0.1. Report if the solution was found and the average time it takes to find a solution (or report no solution). 

In [37]:
import random

def calculate_conflicts(state):
    conflicts = 0
    n = len(state)
    
    for col1 in range(n):
        row1 = state[col1]
        for col2 in range(col1 + 1, n):
            row2 = state[col2]
            if row1 == row2 or col1 == col2 or abs(row1 - row2) == abs(col1 - col2):
                conflicts += 1
    return conflicts

def print_board(state, n):
    board = [["." for _ in range(n)] for _ in range(n)]
    for col, row in enumerate(state):
        board[row][col] = "Q"
    
    for row in board:
        print(" ".join(row))
    print()

def random_individual(n):
    return [random.randint(0, n - 1) for _ in range(n)]

def tournament_selection(population, fitness, k = 3):
    tournament = random.sample(population, k)
    return min(tournament, key=lambda x: fitness(x))

def crossover(parent1, parent2):
    n = len(parent1)
    crossover_point = random.randint(0, n - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

def mutate(individual, mutation_rate):
    for col in range(len(individual)):
        if random.random() < mutation_rate:
            individual[col] = random.randint(0, len(individual) - 1)

def genetic_algorithm(n=4, population_size=10, generations=1000, mutation_rate=0.1, time_start = -1, time_limit = 120, verbose=False):
    # Step 1: Initialize the population
    population = [random_individual(n) for _ in range(population_size)]
    
    if verbose:
        print("Initial population:")
        for i, individual in enumerate(population):
            print(f"Individual {i + 1}:")
            print_board(individual, n)
        print()
    
    for generation in range(generations):
        # Step 2: Calculate fitness (number of conflicts) for each individual
        fitness = [(individual, calculate_conflicts(individual)) for individual in population]
        
        # Step 3: Check if any individual has zero conflicts (solution found)
        for individual, conflicts in fitness:
            if conflicts == 0:
                if verbose:
                    print(f"Solution found after {generation + 1} generations:")
                    print_board(individual, n)
                return individual
        
        if time_start != - 1:
            if abs(time_start - time.time()) > time_limit:
                return None
        
        # Step 4: Sort by fitness (fewest conflicts)
        fitness.sort(key=lambda x: x[1])  # Sort by conflicts
        if verbose:
            print(f"Generation {generation + 1}:")
            for i, (individual, conflicts) in enumerate(fitness):
                print(f"Individual {i + 1}:")
                print_board(individual, n)
                print(f"Conflicts: {conflicts}")
            print()
        
        # Step 5: Select the best individuals to breed
        next_generation = []
        for _ in range(population_size // 2):
            parent1 = tournament_selection(population, calculate_conflicts) # Select with tournament selection k setted by default as 3
            parent2 =  tournament_selection(population, calculate_conflicts)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent1, parent2)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            next_generation.extend([child1, child2])
        
        # Ensure the population size remains constant
        population = next_generation[:population_size]
    
    if verbose:
        print("No solution found after maximum generations.")
    return None

# Solve the 4-queens problem using a Genetic Algorithm with verbose logging
solution = genetic_algorithm(n=4, population_size=10, generations=1000, mutation_rate=0.1, verbose=True)


Initial population:
Individual 1:
. Q . .
. . . Q
Q . . .
. . Q .

Individual 2:
. Q . .
Q . Q .
. . . .
. . . Q

Individual 3:
. Q . .
Q . . Q
. . Q .
. . . .

Individual 4:
Q . . .
. . . .
. Q Q Q
. . . .

Individual 5:
. . Q Q
. . . .
. . . .
Q Q . .

Individual 6:
. . Q Q
. . . .
Q . . .
. Q . .

Individual 7:
. Q . .
Q . Q .
. . . Q
. . . .

Individual 8:
. . Q Q
. Q . .
. . . .
Q . . .

Individual 9:
. . Q .
. . . Q
. Q . .
Q . . .

Individual 10:
. . Q .
. . . .
. Q . .
Q . . Q


Solution found after 1 generations:
. Q . .
. . . Q
Q . . .
. . Q .



*Observation*: While BFS search is complete and none of the other techniques are not, significant computational time and memory is saved by local search techniques. 

### Exercise 5: 
Set a time-out of 2min. runtime, and inspect which is the largest n for finding a solution for each of the methods (BFS/Hill/simulated annealing/k-beams/genetic search). 

In [38]:
n = 4
start_time = time.time()
while abs(start_time- time.time()) < 120:
    solution = bfs_n_queens(n, time_start=start_time, verbose=False)
    n = n + 1*(n != None)
print(f"bfs : {n}") 

n = 4
start_time = time.time()
while abs(start_time- time.time()) < 120:
    solution = hill_climbing_n_queens(n, time_start=start_time,  verbose=False)
    n = n + 1*(n != None)
print(f"hill climbing : {n}") 

n = 4
start_time = time.time()
while abs(start_time- time.time()) < 120:
    solution = local_beam_search(n, k= (2 + n//4), max_iterations=1000*(n//4), time_start=start_time, verbose=False)
    n = n + 1*(solution != None)
print(f"k beans : {n}") 

n = 4
start_time = time.time()
while abs(start_time- time.time()) < 120:
    solution = simulated_annealing_n_queens(n, max_iterations=1000*n, initial_temperature=100*(n/4), cooling_rate=0.98 - (n/400), time_start=start_time, verbose=False)
    n = n + 1*(solution != None)
print(f"simulated anealing : {n}") 

n = 4
start_time = time.time()
while abs(start_time- time.time()) < 120:
    solution = genetic_algorithm(n, population_size=10*(n//4), generations=1000, mutation_rate=1/n, time_start=start_time, verbose=False)
    n = n + 1*(solution != None)
print(f"genetic algorithm : {n}") 


bfs : 15
hill climbing : 56
k beans : 7
simulated anealing : 61
genetic algorithm : 25
